<h3>NFL Game Simulation - Basic Example</h3>

<p>
  <b>Description</b>: Project win percentages based on Points For / Against from the Regular Season.
</p>

<p>
  <b>Question(s)</b>: Which team has the better chance of winning the game?
</p>

<br />
<br />
<br />

In [1]:
import random

import pandas as pd
from scipy.stats import norm

In [2]:
year = 2020
games = pd.read_csv(f'../../data/nfl/pfr-{year}-games.csv', index_col=0)

weeks = [str(i+1) for i in range(19)]
regular_season = games[games.week.isin(weeks)].copy()
ratings = regular_season.groupby('team').mean()[['pf', 'pa']]

ratings.head()

,pf,pa
team,,
Arizona Cardinals,25.6250,22.9375
Atlanta Falcons,24.7500,25.8750
Baltimore Ravens,29.2500,18.9375
Buffalo Bills,31.3125,23.4375
Carolina Panthers,21.8750,25.1250


In [3]:
avg_points = regular_season['pf'].mean()
score_std = regular_season['pf'].std()

def play_game(t1, t2):

  t1_pf, t1_pa = ratings.loc[t1][['pf', 'pa']]
  t2_pf, t2_pa = ratings.loc[t2][['pf', 'pa']]

  team1_mu = (t1_pf * t2_pa) / avg_points
  team2_mu = (t2_pf * t1_pa) / avg_points

  return (
    round(norm.ppf(random.random(), team1_mu, score_std), 2),
    round(norm.ppf(random.random(), team2_mu, score_std), 2),
  )

playoff_schedule = [
  ## round 1
  ('Tampa Bay Buccaneers', 'Washington Football Team'),
  ('New Orleans Saints', 'Chicago Bears'),
  ('Seattle Seahawks', 'Los Angeles Rams'),

  ## round 2
  ('New Orleans Saints', 'Tampa Bay Buccaneers'),
  ('Green Bay Packers', 'Los Angeles Rams'),

  ## round 3
  ('Green Bay Packers', 'Tampa Bay Buccaneers'),
]

simulations = 10000

print('-' * 100)

for team1, team2 in playoff_schedule:

  scores = [
    play_game(team1, team2)
    for _ in range(simulations)
  ]

  t1_wins = sum(
    list(map(lambda sc: sc[0] > sc[1], scores))
  )

  t1_win_percentage = t1_wins / simulations
  t2_win_percentage = 1 - t1_win_percentage

  print(f'{team1}: {round(t1_win_percentage * 100, 2)}')
  print(f'{team2}: {round(t2_win_percentage * 100, 2)}')
  print('-' * 100)

----------------------------------------------------------------------------------------------------
Tampa Bay Buccaneers: 68.62
Washington Football Team: 31.38
----------------------------------------------------------------------------------------------------
New Orleans Saints: 72.5
Chicago Bears: 27.5
----------------------------------------------------------------------------------------------------
Seattle Seahawks: 48.06
Los Angeles Rams: 51.94
----------------------------------------------------------------------------------------------------
New Orleans Saints: 51.5
Tampa Bay Buccaneers: 48.5
----------------------------------------------------------------------------------------------------
Green Bay Packers: 56.48
Los Angeles Rams: 43.52
----------------------------------------------------------------------------------------------------
Green Bay Packers: 49.8
Tampa Bay Buccaneers: 50.2
-----------------------------------------------------------------------------------------